<a href="https://colab.research.google.com/github/ssundar6087/Machine-Learning-Implementations/blob/main/01_K_Means_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from scipy.linalg import norm

In [ ]:
# Constants
RANDOM_SEED = 342
N_SAMPLES = 1000
N_FEATURES = 2
N_CENTERS = 4

In [ ]:
X, y = make_blobs(n_samples=N_SAMPLES, n_features=N_FEATURES, centers=N_CENTERS, random_state=RANDOM_SEED)

In [ ]:
print(X.shape, y.shape)

In [ ]:
print(np.min(y), np.max(y))

In [ ]:
print(X[:5], y[:5])

In [ ]:
# plot the data
def plot_clusters(X, labels, k):
  colors = ['r', 'g', 'b', 'y', 'm']
  plt.figure(figsize=(8,8))
  for c in range(k):
    c_label = labels == c # Get me all points whose label equals c 
    plt.scatter(X[c_label, 0], X[c_label, 1], color=colors[c], marker='.', s=15)

In [ ]:
plot_clusters(X, y, N_CENTERS)

In [ ]:
# Steps in K-means
#1. Initialize centroids randomly 
#2. Assign each data point to closest centroid
#3. Update centroid positions
#4. Repeat 2 & 3 until convergence

In [ ]:
a = range(10)

In [ ]:
random_values = np.random.choice(a, size=N_CENTERS, replace=False)

In [ ]:
random_values

In [ ]:
centroid_idxs = np.random.choice(range(N_SAMPLES), size=N_CENTERS, replace=False)

In [ ]:
centroid_idxs

In [ ]:
X[centroid_idxs]

In [ ]:
def plot_clusters_with_centers(X, labels, centers, k):
  colors = ['r', 'g', 'b', 'y', 'm']
  plt.figure(figsize=(8,8))
  for c in range(k):
    c_label = labels == c # Get me all points whose label equals c 
    plt.scatter(X[c_label, 0], X[c_label, 1], color=colors[c], marker='.', s=15, alpha=0.3)
  plt.scatter(centers[:, 0], centers[:, 1], color='c', marker='x', s=100)

In [ ]:
plot_clusters_with_centers(X, y, X[centroid_idxs], N_CENTERS)

In [ ]:
# 2. Compute distance of each point to all centroids and assign to closest centroid
# Euclidean distance
a = np.arange(0, 8).reshape(-1, 2)

In [ ]:
a

In [ ]:
b = np.array([1, 1])

In [ ]:
((a[0, 0] - b[0])**2 + (a[0, 1] - b[1])**2)**0.5

In [ ]:
((a[1, 0] - b[0])**2 + (a[1, 1] - b[1])**2)**0.5

In [ ]:
norm((a - b), axis=1)

In [ ]:
np.sum(a, axis=0)

In [ ]:
np.sum(a, axis=1)

In [ ]:
centroids = X[centroid_idxs]

In [ ]:
distances = np.empty(shape=(N_SAMPLES, N_CENTERS))
for i, centroid in enumerate(centroids):
  distance = np.array(norm(X - centroid, axis=1))
  distances[:,i] = distance

In [ ]:
distances[:5]

In [ ]:
distances.shape

In [ ]:
# Problem of scale! 
# 10000, 25, 8
# 40000, 40, 10
# Normalize features before fitting model

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
assert X_scaled.shape == X.shape

In [ ]:
X_scaled[:5], X[:5]

In [ ]:
centroids

In [ ]:
centroids = X_scaled[centroid_idxs]

In [ ]:
centroids

In [ ]:
# Finding the closest centroid for each point
for i, point in enumerate(X_scaled):
  distances = [norm(point - centroid) for centroid in centroids]
  print(distances)
  print(np.argmin(distances)) # Returns the minimum value's index along the specified axis
  print(centroids[np.argmin(distances),:])

  if i == 3:
    break

In [ ]:
def assign_clusters(X, centroids):
  n_samples = X.shape[0]
  k = len(centroids)
  distances = np.empty((n_samples, k))
  for i, centroid in enumerate(centroids):
    distances[:, i] = np.array(norm(X - centroid, axis=1))
  closest_centroid = np.argmin(distances, axis=1) # Minimum along each row
  return closest_centroid 

In [ ]:
labels_for_data = assign_clusters(X_scaled, centroids)

In [ ]:
labels_for_data.shape

In [ ]:
labels_for_data[:10]

In [ ]:
# 3. Update centroids based on cluster labels
new_centroids = []
for c in range(N_CENTERS):
  cluster_data = X_scaled[labels_for_data == c] # pick all points with label == c
  new_centroids.append(np.mean(cluster_data, axis=0))
  

In [ ]:
np.array(new_centroids).shape

In [ ]:
new_centroids

In [ ]:
centroids

In [ ]:
# Step 4: Loop steps 2 & 3 until convergence

In [ ]:
from numpy.lib.function_base import kaiser
# Putting it all together
class KMeans():
  def __init__(self, k, tolerance, max_iters):
    self.k = k 
    self.tolerance = tolerance 
    self.max_iters = max_iters 
    self.inertia = 0.0 # Evaluating K-means
    self.centroids = []
  
  # Step 1: Init Centroids
  def init_centroids(self, X):
    n_samples = X.shape[0]
    centroid_idxs = np.random.choice(range(n_samples), size=self.k, replace=False)
    return X[centroid_idxs]
  
  # Step 2: Assign points to clusters
  def assign_clusters(self, X):
    n_samples = X.shape[0] 
    distances = np.empty((n_samples, self.k))
    for c in range(self.k):
      distances[:, c] = norm(X - self.centroids[c], axis=1)
    labels = np.argmin(distances, axis=1)
    return labels 
  
  # Step 3: Update centroids
  def update_centroids(self, X, labels):
    new_centroids = []
    for c in range(self.k):
      cluster_data = X[labels == c]
      new_centroids.append(np.mean(cluster_data, axis=0))
    return np.array(new_centroids)
  
  # Measure K-means performance
  def compute_inertia(self, X, labels):
    for c in range(self.k):
      cluster_data = X[labels == c]
      within_cluster_distance = np.sum(norm(cluster_data - self.centroids[c], axis=1)**2)
      self.inertia += within_cluster_distance
    return self.inertia 
  
  # Combine all steps 
  def fit(self, X):
    self.centroids = self.init_centroids(X)
    for i in range(self.max_iters):
      cluster_labels = self.assign_clusters(X)
      prev_centroids = self.centroids # Save to calculate displacement
      self.centroids = self.update_centroids(X, cluster_labels)

      # Check if converged
      displacement = 0.0
      for c in range(self.k):
        displacement += norm(prev_centroids[c] - self.centroids[c])
      
      if displacement < self.tolerance:
        print(f"Converged in {i + 1} iterations\n")
        self.compute_inertia(X, cluster_labels)
        return self.centroids, cluster_labels, self.inertia 
    
    self.compute_inertia(X, cluster_labels)
    return self.centroids, cluster_labels, self.inertia 
    


In [ ]:
my_kmeans = KMeans(k=N_CENTERS, tolerance=1e-4, max_iters=300)

In [ ]:
final_centroids, final_labels, inertia = my_kmeans.fit(X_scaled) # Send scaled data :)

In [ ]:
print(final_centroids, inertia)

In [ ]:
plot_clusters_with_centers(X_scaled, final_labels, final_centroids, k=N_CENTERS)

In [ ]:
# Elbow Rule: How to find K??
inertia_scores = []
for kk in range(2, 10):
  kmeans_obj = KMeans(k=kk, tolerance=1e-4, max_iters=300)
  run_centroids, run_labels, run_inertia = kmeans_obj.fit(X_scaled)
  inertia_scores.append(run_inertia)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(range(2, 10), inertia_scores, c='r', marker='x', s=100)
plt.plot(range(2, 10), inertia_scores)
plt.xlabel('k')
plt.ylabel('inertia')

In [ ]:
# Compare with SK-learn's KMeans
from sklearn.cluster import KMeans
sk_kmeans = KMeans(n_clusters=N_CENTERS)
clusters = sk_kmeans.fit_transform(X_scaled)

In [ ]:
clusters.shape

In [ ]:
sk_kmeans.inertia_

In [ ]:
centers = sk_kmeans.cluster_centers_ 
plt.figure(figsize=(8,8))
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], s=10, c=sk_kmeans.labels_)
plt.scatter(centers[:, 0], centers[:, 1], c='r', s=20)

In [ ]:
# Fin :D